In [1]:
import rebound
import numpy as np
from pathlib import Path
import sys
sys.path.insert(0, '../src')
import os
import horizons_api
import pandas as pd

astdys = pd.read_csv('astdys_tnos.csv')
clones = 0 
#for i in range(len(astdys)):
#for i in range(10):
#objnum = int(sys.argv[1])
#print(objnum)
vals = np.zeros((1,8))
plan_vals = np.zeros((1,64))

plan_cols = []
for i in range(8):
   plan_cols.append('mass_'+str(i+1))
   plan_cols.append('radius_'+str(i+1))
   plan_cols.append('x_'+str(i+1))
   plan_cols.append('y_'+str(i+1))
   plan_cols.append('z_'+str(i+1))
   plan_cols.append('vx_'+str(i+1))
   plan_cols.append('vy_'+str(i+1))
   plan_cols.append('vz_'+str(i+1))


#vals = [flag,epoch,sbx,sby,sbz,sbvx,sbvy,sbvz]
horizon_data = pd.DataFrame(vals,columns=['flag','epoch','sbx','sby','sbz','sbvx','sbvy','sbvz'])
horizon_planets = pd.DataFrame(plan_vals, columns=plan_cols)
planet_id = {1: 'mercury', 2: 'venus', 3:'earth', 4:'mars', 5: 'jupiter', 6 : 'saturn', 7 : 'uranus', 8 : 'neptune'}

objs = ['1999 CB119','1999 CC119','2003 FA130','2003 FD128']
for i in range(len(objs)):
#for i in range(2):
    print(i)
    objname = objs[i]
    #objname = astdys['Name'].iloc[i]
    filename = 'TNOs/' + objname
    if not os.path.isdir(filename):
        os.mkdir(filename)
    sbody = objname    
    des=sbody
    ntp = 1 + clones
    sbx = np.zeros(ntp)
    sby = np.zeros(ntp)
    sbz = np.zeros(ntp)
    sbvx = np.zeros(ntp)
    sbvy = np.zeros(ntp)
    sbvz = np.zeros(ntp)

    flag, epoch, sbx, sby, sbz, sbvx, sbvy, sbvz = horizons_api.query_sb_from_jpl(des=des,clones=clones)
    print(flag, epoch, sbx, sby, sbz)
    horizon_data['flag'][0] = flag
    horizon_data['epoch'][0] = epoch
    horizon_data['sbx'][0] = sbx
    horizon_data['sby'][0] = sby
    horizon_data['sbz'][0] = sbz
    horizon_data['sbvx'][0] = sbvx
    horizon_data['sbvy'][0] = sbvy
    horizon_data['sbvz'][0] = sbvz
    
    horizon_data.to_csv('test_jpl.csv')  


    notplanets = [1,2,3,4]
    planets = [5,6,7,8]
                

    for pl in notplanets:
        flag, mass, radius, [x, y, z], [vx, vy, vz] = horizons_api.query_horizons_planets(obj=planet_id[pl],epoch=epoch)
        horizon_planets['mass_'+str(pl)] = mass
        horizon_planets['radius_'+str(pl)] = radius
        horizon_planets['x_'+str(pl)] = x
        horizon_planets['y_'+str(pl)] = y
        horizon_planets['z_'+str(pl)] = z
        horizon_planets['vx_'+str(pl)] = vx
        horizon_planets['vy_'+str(pl)] = vy
        horizon_planets['vz_'+str(pl)] = vz

    for pl in planets:
        flag, mass, radius, [x, y, z], [vx, vy, vz] = horizons_api.query_horizons_planets(obj=planet_id[pl], epoch=epoch)
        horizon_planets['mass_'+str(pl)] = mass
        horizon_planets['radius_'+str(pl)] = radius
        horizon_planets['x_'+str(pl)] = x
        horizon_planets['y_'+str(pl)] = y
        horizon_planets['z_'+str(pl)] = z
        horizon_planets['vx_'+str(pl)] = vx
        horizon_planets['vy_'+str(pl)] = vy
        horizon_planets['vz_'+str(pl)] = vz

    horizon_planets.to_csv('testing_jpl.csv')
    


0
unable to pull the GM value from the horizons results, which are:

*******************************************************************************
 Multiple major-bodies match string "1999"

  ID#      Name                               Designation  IAU/aliases/other   
  -------  ---------------------------------- -----------  ------------------- 
 20612095  1999 OJ4 (system barycenter)                                         
920612095  1999 OJ4 (primary body)                                              
120612095  1999 OJ4 1                                                           
 
   Number of matches =  3. Use ID# to make unique selection.
*******************************************************************************

0 0.0 0.0 0.0 0.0
1
unable to pull the GM value from the horizons results, which are:

*******************************************************************************
 Multiple major-bodies match string "1999"

  ID#      Name                               Des

In [57]:
from astroquery.jplsbdb import SBDB
import requests
import json
des = '1999 CB119'
obj= SBDB.query(des,id_type='search',full_precision=True,covariance='mat',phys=True)
obj_dat=obj['object']
spkid = obj_dat['spkid']
print(spkid)
url = 'https://ssd.jpl.nasa.gov/api/horizons.api'
start_time = 'JD'+str(epoch)
stop_time = 'JD'+str(epoch+1)
url += "?format=json&EPHEM_TYPE=ELEMENTS&OBJ_DATA=YES&CENTER='@Sun'&OUT_UNITS='AU-D'"
#url += "&DES=  "
url += "&COMMAND='DES%3D"+str(spkid)+ "%3B'" +"&START_TIME=" +start_time + "&STOP_TIME=" + stop_time
    
    #run the query and exit if it fails
response = requests.get(url)
data = json.loads(response.text)
print(data)

3031828
{'signature': {'source': 'NASA/JPL Horizons API', 'version': '1.2'}, 'error': 'No ephemeris for target "(1999 CB119)" prior to A.D. 1599-DEC-11 00:00:00.0000 TDB', 'result': '*******************************************************************************\nJPL/HORIZONS                    (1999 CB119)               2022-Nov-16 10:12:10\nRec #:50001765 (+COV) Soln.date: 2022-Apr-28_20:45:05     # obs: 30 (1999-2022)\n \nIAU76/J2000 helio. ecliptic osc. elements (au, days, deg., period=Julian yrs):\n \n  EPOCH=  2454859.5 ! 2009-Jan-28.00 (TDB)         Residual RMS= .17263\n   EC= .1270972682599154   QR= 40.88860966064035   TP= 2449400.2592243166\n   OM= 168.1339990130917   W=  350.9277828607575   IN= 8.756492930979775\n   A= 46.84211444628098    MA= 16.78346342992745   ADIST= 52.79561923192161\n   PER= 320.59957          N= .003074322           ANGMOM= .116778613\n   DAN= 40.94637           DDN= 52.69963           L= 159.1657967\n   B= -1.3754807           MOID= 39.89820099       

In [21]:
import pandas as pd
data = pd.read_csv('data_files/Distant_2.txt')
print(data)
names = []
for i in range(len(data)):
    #print(len(data))
    line = data.iloc[i][0]
    #print(line)
    name = line[175:194].strip()
    names.append(name)

     00944   10.55  0.15 K232P 113.66620   56.49903   21.44365   42.57966  0.6615382  0.07187610   5.7290115  0 E2023-M50  1979  30 1920-2023 0.59 M-v 3Ek MPCLINUX   0000    (944) Hidalgo            20230618
0     02060    5.64  0.15 K232P 192.89992  339.43906...                                                                                                                                                        
1     05145    7.07  0.15 K232P 124.02785  354.62199...                                                                                                                                                        
2     05335   13.23  0.15 K232P 285.39741  191.11183...                                                                                                                                                        
3     07066    9.57  0.15 K232P  91.82203  170.77708...                                                                                                                 

In [24]:
print(names)
namedf = pd.DataFrame(names)
namedf.to_csv('data_files/Distant_names.csv')

['Chiron', 'Pholus', 'Damocles', 'Nessus', 'Asbolus', 'Chariklo', 'Hylonome', '1999 RG33', 'Albion', '1993 SB', '1993 SC', '1994 GV9', '1994 JS', 'Arawn', '1994 TB', '1995 DA2', '1996 TL66', '1996 TP66', '1997 CR29', '1994 JQ1', '1994 VK8', '1996 SZ4', '1996 TO66', 'Chaos', 'Varuna', '1995 QZ9', '1996 TR66', 'Dioretsa', '1995 SM55', '1997 QJ4', '1998 HJ151', '1996 GQ21', '1998 SM165', '1999 DE9', 'Ixion', '1999 TD10', 'Elatus', 'Thereus', '1995 QY9', '1997 CU29', '1998 BU48', '1998 VG44', '1998 SN165', 'Narcissus', 'Rhadamanthus', '1999 HB12', 'Huya', '1999 KR16', '2001 UR163', 'Typhon', '1999 OX3', '2000 PV29', 'Lempo', '2000 GN171', '1995 TL8', 'Pelion', '1999 RA215', 'Quaoar', '1998 HM151', 'Okyrhoe', 'Cyllarus', 'Deucalion', '2000 PJ30', 'Bienor', '2002 AW197', 'Amycus', '2002 TX300', '2002 UX25', '2002 VE95', 'Logos', '1999 CL158', '2000 CH105', '2000 CM114', 'Echeclus', '2000 EE173', '2000 FD8', '2000 FE8', '2001 BL41', '2002 RP120', 'Ceto', '1999 OF4', 'Borasisi', '1998 WW24', '